### Задание
Вам необходимо провести эксперименты по начальному обучению различных моделей и сравнить результаты.

1. Возьмите датасет EMNIST из torchvision
1. Обучите на нём модели (с нуля по 10 эпох):
    * ResNet 18,
    * VGG 16,
    * Inception v3,
    * DenseNet 161.
1. Сведите результаты обучения моделей (графики лоса) в таблицу и сравните их.

In [24]:
from collections import defaultdict

In [2]:
dd_list = defaultdict(int)
dd_list['key1']=0
print(dd_list,dd_list['key1'],dd_list['key2'],dd_list.get('key2'))

defaultdict(<class 'int'>, {'key1': 0, 'key2': 0}) 0 0 0


In [ ]:
dd_list = defaultdict(list)
dd_list['key1'].append(1)
print(dd_list,dd_list['key1'],dd_list['key2'])

defaultdict(<class 'list'>, {'key1': [1], 'key2': []}) [1] []


In [ ]:
dd_list = defaultdict(lambda:defaultdict(lambda:defaultdict(list)))
dd_list['key1']['jey1']['hey1'].append(1)
print(dd_list,'\n',dd_list['key1']['jey1']['hey1'],'\n',dd_list['key2']['jey2']['hey2'],'\n',
      dd_list['key1']['jey1'],'\n',dd_list['key2']['jey2'],'\n',
      dd_list['key1'],'\n',dd_list['key2'],'\n')

defaultdict(<function <lambda> at 0x7e3f851d2dd0>, {'key1': defaultdict(<function <lambda>.<locals>.<lambda> at 0x7e3f851d2cb0>, {'jey1': defaultdict(<class 'list'>, {'hey1': [1]})}), 'key2': defaultdict(<function <lambda>.<locals>.<lambda> at 0x7e3f851d2710>, {'jey2': defaultdict(<class 'list'>, {'hey2': []})})}) 
 [1] 
 [] 
 defaultdict(<class 'list'>, {'hey1': [1]}) 
 defaultdict(<class 'list'>, {'hey2': []}) 
 defaultdict(<function <lambda>.<locals>.<lambda> at 0x7e3f851d2cb0>, {'jey1': defaultdict(<class 'list'>, {'hey1': [1]})}) 
 defaultdict(<function <lambda>.<locals>.<lambda> at 0x7e3f851d2710>, {'jey2': defaultdict(<class 'list'>, {'hey2': []})}) 



In [25]:
import time
import pandas as pd
import torchvision as tv
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from torchsummary import summary
%matplotlib inline

In [26]:



BATCH_SIZE = 256

transforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST(
    '.', split='mnist', train=True, transform=transforms, download=True)
test_dataset = tv.datasets.EMNIST(
    '.', split='mnist', train=False, transform=transforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [27]:
history = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
list_types_NN=['ResNet18','VGG16','Inception_v3','DenseNet161']

In [6]:
def plot_learning_curves(history):
    '''
    Функция для вывода графиков лосса и метрики во время обучения.
    '''
    fig = plt.figure(figsize=(20, 20))

    for i,type_NN in enumerate(list_types_NN):
        plt.subplot(4,2,i)
        plt.title(f'Loss_{type_NN}', fontsize=15)
        plt.plot(history[type_NN]['loss']['train'], label='train')
        plt.plot(history[type_NN]['loss']['val'], label='val')
        plt.ylabel('LOSS', fontsize=15)
        plt.xlabel('EPOCH', fontsize=15)
        plt.legend()

        plt.subplot(4,2,i+1)
        plt.title('Accuracy_{type_NN}', fontsize=15)
        plt.plot(history[type_NN]['acc']['train'], label='train')
        plt.plot(history[type_NN]['acc']['val'], label='val')
        plt.ylabel('LOSS', fontsize=15)
        plt.xlabel('EPOCH', fontsize=15)
        plt.legend()
    plt.show()

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


#### ResNet 18

In [ ]:
weights='IMAGENET1K_V1'
model = tv.models.resnet18(weights=weights)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 210MB/s]


In [ ]:
'''from torchvision.models import resnet18, ResNet18_Weights

weights = ResNet18_Weights.DEFAULT #weights=ResNet50_Weights.DEFAULT
model = resnet18(weights=weights)'''

'from torchvision.models import resnet18, ResNet18_Weights\n\nweights = ResNet18_Weights.DEFAULT #weights=ResNet50_Weights.DEFAULT\nmodel = resnet18(weights=weights)'

In [ ]:
#from torchvision.models import resnet18, ResNet18_Weights
'''
model = tv.models.resnet18()
model.load_state_dict(torch.load('resnet18-f37072fd.pth'))'''

<All keys matched successfully>

In [ ]:
model = model.to(device)
model

In [ ]:
summary(model.to(device), input_size=(3, 224, 224))

In [6]:
def evaluate_accuracy(data_iter, net, name_NN):
    loss = nn.CrossEntropyLoss(reduction='sum')
    acc_sum, n, test_loss = 0, 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        '''if name_NN==list_types_NN[2]:
          y_pred = net(X)
        else:
          y_pred = net(X)'''
        #print(net(X).flatten())
        #print(y)
        y_pred = net(X)
        l = loss(y_pred, y)
        test_loss += l.item()

        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]

    history[name_NN]['loss']['val'].append(test_loss/n)
    history[name_NN]['acc']['val'].append(acc_sum.item()/n)
    return acc_sum.item() / n, test_loss/n

In [7]:
def train(net, train_iter, test_iter, trainer, num_epochs, name_NN):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()

            '''if name_NN==list_types_NN[2]:
              y_hat,aux = net(X)
            else:
              y_hat = net(X)'''
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 50 == 0:
                print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")

        history[name_NN]['loss']['train'].append(train_l_sum / n)
        history[name_NN]['acc']['train'].append(train_acc_sum / n)

        test_acc, test_loss = evaluate_accuracy(test_iter, net.to(device),name_NN)

        print('-' * 20)
        print(f'epoch {epoch + 1}, testloss {test_loss:.4f}, trainloss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=10)

In [ ]:
model.fc

Linear(in_features=512, out_features=10, bias=True)

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 3, list_types_NN[0])

Step 0. time since epoch: 0.806. Train acc: 0.074. Train Loss: 2.445
Step 50. time since epoch: 34.796. Train acc: 0.681. Train Loss: 1.319
Step 100. time since epoch: 67.592. Train acc: 0.787. Train Loss: 0.946
Step 150. time since epoch: 100.039. Train acc: 0.832. Train Loss: 0.764
Step 200. time since epoch: 132.193. Train acc: 0.858. Train Loss: 0.653
--------------------
epoch 1, testloss 0.2608, trainloss 0.6004, train acc 0.870, test acc 0.945, time 178.6 sec
Step 0. time since epoch: 0.601. Train acc: 0.914. Train Loss: 0.305
Step 50. time since epoch: 30.039. Train acc: 0.945. Train Loss: 0.250
Step 100. time since epoch: 58.777. Train acc: 0.948. Train Loss: 0.237
Step 150. time since epoch: 87.644. Train acc: 0.949. Train Loss: 0.228
Step 200. time since epoch: 117.269. Train acc: 0.951. Train Loss: 0.219
--------------------
epoch 2, testloss 0.1790, trainloss 0.2142, train acc 0.952, test acc 0.959, time 159.8 sec
Step 0. time since epoch: 0.599. Train acc: 0.930. Train Lo

In [ ]:
history

defaultdict(<function __main__.<lambda>()>,
            {'ResNet18': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'loss': defaultdict(list,
                                      {'train': [0.600373122437795,
                                        0.21424450279871624,
                                        0.1620264399210612],
                                       'val': [0.2607715274810791,
                                        0.1789951827645302,
                                        0.14773887801766394]}),
                          'acc': defaultdict(list,
                                      {'train': [0.8696,
                                        0.9516333333333333,
                                        0.9600666666666666],
                                       'val': [0.9454, 0.9591, 0.9636]})})})

In [ ]:
plot_learning_curves(history)

#### VGG16

In [ ]:

model = tv.models.vgg16(weights='IMAGENET1K_V1')

'''model = tv.models.vgg16()
model.load_state_dict(torch.load('vgg16-397923af.pth'))'''

"model = tv.models.vgg16()\nmodel.load_state_dict(torch.load('vgg16-397923af.pth'))"

In [ ]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
summary(model.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [ ]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=10)
model.classifier[6]

Linear(in_features=4096, out_features=10, bias=True)

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [ ]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=10, bias=True)
)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 3, list_types_NN[1])

Step 0. time since epoch: 1.791. Train acc: 0.121. Train Loss: 2.372
Step 50. time since epoch: 79.629. Train acc: 0.690. Train Loss: 1.105
Step 100. time since epoch: 157.412. Train acc: 0.764. Train Loss: 0.841
Step 150. time since epoch: 235.578. Train acc: 0.796. Train Loss: 0.719
Step 200. time since epoch: 313.545. Train acc: 0.815. Train Loss: 0.647
--------------------
epoch 1, testloss 0.2923, trainloss 0.6122, train acc 0.824, test acc 0.934, time 458.2 sec
Step 0. time since epoch: 1.693. Train acc: 0.941. Train Loss: 0.273
Step 50. time since epoch: 79.749. Train acc: 0.940. Train Loss: 0.269
Step 100. time since epoch: 157.539. Train acc: 0.943. Train Loss: 0.255
Step 150. time since epoch: 235.711. Train acc: 0.943. Train Loss: 0.244
Step 200. time since epoch: 313.775. Train acc: 0.945. Train Loss: 0.233
--------------------
epoch 2, testloss 0.1881, trainloss 0.2276, train acc 0.946, test acc 0.954, time 458.2 sec
Step 0. time since epoch: 1.532. Train acc: 0.961. Train

#### Inception v3

In [ ]:
model = tv.models.inception_v3(weights='IMAGENET1K_V1')
'''model = tv.models.inception_v3()
model.load_state_dict(torch.load('inception_v3_google-0cc3c7bd.pth'))'''

"model = tv.models.inception_v3()\nmodel.load_state_dict(torch.load('inception_v3_google-0cc3c7bd.pth'))"

In [ ]:
model.aux_logits=False

In [ ]:
model.aux_logits

False

In [ ]:
BATCH_SIZE = 256

transforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST(
    '.', split='mnist', train=True, transform=transforms, download=True)
test_dataset = tv.datasets.EMNIST(
    '.', split='mnist', train=False, transform=transforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
summary(model.to(device), input_size=( 3,299,299))

In [ ]:
for param in model.parameters():
    param.requires_grad = False
model.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [ ]:
model.fc=nn.Linear(in_features=2048, out_features=10)
model.fc

Linear(in_features=2048, out_features=10, bias=True)

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

This network is unique because it has two output layers when training. The primary output is a linear layer at the end of the network. The second output is known as an auxiliary output and is contained in the AuxLogits part of the network.

In [ ]:
train(model, train_iter, test_iter, trainer, 3, list_types_NN[2])

Step 0. time since epoch: 1.645. Train acc: 0.090. Train Loss: 2.351
Step 50. time since epoch: 84.982. Train acc: 0.591. Train Loss: 1.521
Step 100. time since epoch: 168.223. Train acc: 0.682. Train Loss: 1.203
Step 150. time since epoch: 251.457. Train acc: 0.725. Train Loss: 1.038
Step 200. time since epoch: 335.094. Train acc: 0.749. Train Loss: 0.936
--------------------
epoch 1, testloss 0.4886, trainloss 0.8859, train acc 0.761, test acc 0.875, time 461.6 sec
Step 0. time since epoch: 1.476. Train acc: 0.867. Train Loss: 0.513
Step 50. time since epoch: 78.504. Train acc: 0.887. Train Loss: 0.458
Step 100. time since epoch: 155.389. Train acc: 0.890. Train Loss: 0.440
Step 150. time since epoch: 232.092. Train acc: 0.893. Train Loss: 0.423
Step 200. time since epoch: 308.950. Train acc: 0.894. Train Loss: 0.411
--------------------
epoch 2, testloss 0.3480, trainloss 0.4027, train acc 0.896, test acc 0.908, time 431.5 sec
Step 0. time since epoch: 1.748. Train acc: 0.887. Train

In [ ]:
history

defaultdict(<function __main__.<lambda>()>,
            {'ResNet18': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'loss': defaultdict(list,
                                      {'train': [0.600373122437795,
                                        0.21424450279871624,
                                        0.1620264399210612],
                                       'val': [0.2607715274810791,
                                        0.1789951827645302,
                                        0.14773887801766394]}),
                          'acc': defaultdict(list,
                                      {'train': [0.8696,
                                        0.9516333333333333,
                                        0.9600666666666666],
                                       'val': [0.9454, 0.9591, 0.9636]})}),
             'VGG16': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'loss': defaultdict(l

#### DenseNet161

In [28]:
model = tv.models.densenet161(pretrained=True)
#model = tv.models.densenet161(weights='IMAGENET1K_V1')
#model = tv.models.densenet161()
#model.load_state_dict(torch.load('densenet161-8d451a50.pth'))

#model_dict = torch.load('densenet161-8d451a50.pth')
#model

i:\Users\Konst\anaconda3\envs\my_3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
i:\Users\Konst\anaconda3\envs\my_3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
BATCH_SIZE = 256
transforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST(
    '.', split='mnist', train=True, transform=transforms, download=True)
test_dataset = tv.datasets.EMNIST(
    '.', split='mnist', train=False, transform=transforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
#model = model.to(device)
model

In [ ]:
summary(model.to(device), input_size=(3, 224, 224))

In [32]:
for param in model.parameters():
    param.requires_grad = False
model.classifier

Linear(in_features=2208, out_features=1000, bias=True)

In [33]:
model.classifier=nn.Linear(in_features=2208, out_features=10)
model.classifier

Linear(in_features=2208, out_features=10, bias=True)

In [34]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.weight
	 classifier.bias


In [37]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 3, list_types_NN[3])